In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *

In [2]:
global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
model_name = "VGG16_quant4bit"
model = VGG16_quant()

conv_layer_index = 27

model.features[24] = QuantConv2d(256, 8, kernel_size=3, stride=1, padding=1, bias=False)
model.features[25] = nn.BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
model.features[conv_layer_index] = QuantConv2d(
    in_channels=8,
    out_channels=8,
    kernel_size=3,
    stride=1,
    padding=1,
    bias=False
)
model.features[28] = nn.Sequential()
model.features[30] = QuantConv2d(8, 512, kernel_size=3, stride=1, padding=1, bias=False)

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [2]:
# This cell won't be given, but students will complete the training

lr = 3.95e-2
weight_decay = 1e-4
epochs = 200
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.92, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 3.483 (3.483)	Data 0.968 (0.968)	Loss 2.4381 (2.4381)	Prec 7.812% (7.812%)
Epoch: [0][100/391]	Time 0.043 (0.079)	Data 0.002 (0.013)	Loss 2.3082 (3.5804)	Prec 9.375% (11.409%)
Epoch: [0][200/391]	Time 0.049 (0.062)	Data 0.003 (0.008)	Loss 2.3215 (2.9492)	Prec 14.062% (11.785%)
Epoch: [0][300/391]	Time 0.042 (0.056)	Data 0.003 (0.006)	Loss 2.2700 (2.7269)	Prec 11.719% (12.059%)
Validation starts
Test: [0/79]	Time 0.291 (0.291)	Loss 2.2449 (2.2449)	Prec 14.062% (14.062%)
 * Prec 15.660% 
best acc: 15.660000
Epoch: [1][0/391]	Time 1.131 (1.131)	Data 1.074 (1.074)	Loss 2.3365 (2.3365)	Prec 13.281% (13.281%)
Epoch: [1][100/391]	Time 0.041 (0.052)	Data 0.002 (0.013)	Loss 2.2836 (2.2461)	Prec 18.750% (14.124%)
Epoch: [1][200/391]	Time 0.041 (0.047)	Data 0.002 (0.008)	Loss 2.2234 (2.2306)	Prec 17.188% (15.295%)
Epoch: [1][300/391]	Time 0.044 (0.046)	Data 0.003 (0.006)	Loss 2.1250 (2.2139)	Prec 21.094% (15.900%)
Validation starts
Test: [0/79]	Time 0.309 (0.309)	Loss 1.956

In [3]:
PATH = "result/VGG16_quant4bit/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9181/10000 (92%)



In [4]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
device = torch.device("cuda" if use_gpu else "cpu") 
counter =0
for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):
        print("prehooked")
        counter += 1
        print(layer, counter)
        layer.register_forward_pre_hook(save_output)       ## Input for the module will be grapped       
####################################################

dataiter = iter(trainloader)
images, labels = next(dataiter)
images = images.to(device)
out = model(images)


prehooked
QuantConv2d(
  3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 1
prehooked
QuantConv2d(
  64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 2
prehooked
QuantConv2d(
  64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 3
prehooked
QuantConv2d(
  128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 4
prehooked
QuantConv2d(
  128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 5
prehooked
QuantConv2d(
  256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 6
prehooked
QuantConv2d(
  256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 7
prehooked
QuantConv2d(
  2

In [5]:
weight = model.features[0].weight.abs().sum()
print(weight)

tensor(289.2113, device='cuda:0', grad_fn=<SumBackward0>)


In [10]:
w_bit = 4
weight_q = model.features[0].weight_q # quantized value is stored during the training
w_alpha = model.features[0].weight_quant.wgt_alpha
w_delta = w_alpha/(2**(w_bit-1)-1)
weight_int = weight_q/w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[ 4.,  2.,  0.],
          [ 3., -1., -0.],
          [-1., -3., -3.]],

         [[ 2., -1., -3.],
          [ 2., -2., -2.],
          [-1., -4., -3.]],

         [[ 3.,  1., -2.],
          [ 4.,  2.,  1.],
          [ 1.,  0., -1.]]],


        [[[-0.,  0.,  0.],
          [-1., -1., -0.],
          [-2., -2., -2.]],

         [[ 1.,  1.,  1.],
          [ 0.,  0.,  1.],
          [-1., -2., -2.]],

         [[ 1.,  1.,  1.],
          [ 1.,  1.,  1.],
          [-1., -1., -1.]]],


        [[[-4., -2.,  0.],
          [-6.,  1.,  5.],
          [-1.,  3.,  3.]],

         [[-4., -1.,  4.],
          [-7.,  2.,  7.],
          [-4.,  1.,  4.]],

         [[-3., -1.,  3.],
          [-7.,  1.,  6.],
          [-4., -1.,  2.]]],


        ...,


        [[[-0.,  0.,  0.],
          [-0.,  1.,  1.],
          [ 0.,  0.,  0.]],

         [[-0.,  1.,  1.],
          [-0.,  1.,  1.],
          [-0.,  0.,  1.]],

         [[-0.,  0.,  0.],
          [-1., -1., -0.],
          [-

In [11]:
weight_int.size()

torch.Size([64, 3, 3, 3])

In [12]:
x_bit = 4
x = save_output.outputs[0][0]  # input of the 8th conv layer
x_alpha  = model.features[0].act_alpha
x_delta = x_alpha/(2**x_bit-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q/x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[-14.0000, -14.0000, -14.0000,  ..., -14.0000, -14.0000, -14.0000],
          [-14.0000, -14.0000, -14.0000,  ..., -14.0000, -14.0000, -14.0000],
          [ 15.0000,  15.0000,  15.0000,  ...,  15.0000,  15.0000,  15.0000],
          ...,
          [  2.0000,   3.0000,   4.0000,  ...,   4.0000,   5.0000,   7.0000],
          [  0.0000,   1.0000,   2.0000,  ...,  -1.0000,   2.0000,   6.0000],
          [  7.0000,   7.0000,   7.0000,  ...,   7.0000,   7.0000,   9.0000]],

         [[-14.0000, -14.0000, -14.0000,  ..., -14.0000, -14.0000, -14.0000],
          [-14.0000, -14.0000, -14.0000,  ..., -14.0000, -14.0000, -14.0000],
          [ 15.0000,  15.0000,  15.0000,  ...,  15.0000,  15.0000,  15.0000],
          ...,
          [  3.0000,   4.0000,   5.0000,  ...,   4.0000,   6.0000,   7.0000],
          [  1.0000,   2.0000,   3.0000,  ...,  -1.0000,   2.0000,   6.0000],
          [  7.0000,   8.0000,   8.0000,  ...,   7.0000,   7.0000,   9.0000]],

         [[-12.0000, -12.0000,

In [13]:
x_int.size()

torch.Size([128, 3, 32, 32])

In [16]:
conv_int = torch.nn.Conv2d(in_channels = 3, out_channels=64, kernel_size = 3, padding=1, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
relu = nn.ReLU()
output_int = conv_int(x_int)
output_recovered = output_int*w_delta*x_delta
output_recovered = relu(output_recovered)

In [17]:
difference = abs(save_output.outputs[1][0] - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(1.4798, device='cuda:0', grad_fn=<MeanBackward0>)


In [21]:
x_pad = torch.zeros(128, 3, 34, 34).cuda()

In [22]:
x_pad[ : ,  :, 1:33, 1:33] = x_int.cuda()

In [23]:
x_pad.size()

torch.Size([128, 3, 34, 34])

In [38]:
x_pad[0][2]

tensor([[  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000, -12.0000, -12.0000,  ..., -12.0000, -12.0000,   0.0000],
        [  0.0000, -12.0000, -12.0000,  ..., -12.0000, -12.0000,   0.0000],
        ...,
        [  0.0000,   4.0000,   4.0000,  ...,   3.0000,   6.0000,   0.0000],
        [  0.0000,   8.0000,   8.0000,  ...,   8.0000,   9.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000]],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [24]:
X = x_pad[0]
X = torch.reshape(X, (X.size(0), -1))

In [37]:
X[0][34:68]

tensor([  0., -14., -14., -14., -14., -14., -14., -14., -14., -14., -14., -14.,
        -14., -14., -14., -14., -14., -14., -14., -14., -14., -14., -14., -14.,
        -14., -14., -14., -14., -14., -14., -14., -14., -14.,   0.],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [26]:
X.size() # in_ch = 3, nij = 34 * 34 = 1156

torch.Size([3, 1156])

In [56]:
# output stationary
tile_id = 0 
nij = 200 # just a random number
#X = a_tile[tile_id,:,nij:nij+64]  # [tile_num, array row num, time_steps]

# time0row7 ... time0row0
# ...
# time8row7 ... time0row0

# row 7 ............... row0
#  7  6  5  4  3  2  1  0      time0
#  8  7  6  5  4  3  2  1
# 11 10  9  8  5  4  3  2
# 15 14 13 12  9  8 35 34
# 16 15 14 13 10  9 36 35
# 17 16 15 14 11 10 37 36
# 21 20 19 18 15 14 69 68
# 22 21 20 19 16 15 70 69
# 23 22 21 20 17 16 71 70      time8

bit_precision = 4
row_index = [7, 6, 5, 4, 3, 2, 1, 0]
kij_index = [0, 1, 2, 34, 35, 36, 68, 69, 70]

file = open('activation_os.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for in_ch in range(X.size(0)):  # big time loop - in_ch = 3
    for kij in range(9): # kij 0~8 - time (+ kij_index[0-8])
        for j in range(8): # row 7-0 (row_index[0-7])
            ind = row_index[j]+kij_index[kij]
            if (X[in_ch, ind].item()<0):
                X_bin = '{0:04b}'.format(int(X[in_ch, ind].item()+2**bit_precision+0.001))
            else:
                X_bin = '{0:04b}'.format(int(X[in_ch, ind].item()+0.001))                
            for k in range(bit_precision):
                file.write(X_bin[k])        
            #file.write(' ')  # for visibility with blank between words, you can use
            print(X[in_ch, ind].item(), end=' ')
        file.write('\n')
        print("\n")
file.close() #close file 

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

-14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 0.0 

-14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 

-14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 

-14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 0.0 

-14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 

-14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

-14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 0.0 

-14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 

-14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 

-14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 0.0 

-14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 

-14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 -14.0 

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

-12.000000953674316 -12.000000953674316 -12.000000953674316 -12.000000953674316 -12.000000953674316 -1

In [15]:
# output stationary - separate files
tile_id = 0 
nij = 200 # just a random number
#X = a_tile[tile_id,:,nij:nij+64]  # [tile_num, array row num, time_steps]

# time0row7 ... time0row0
# ...
# time8row7 ... time0row0

# row 7 ............... row0
#  9  8  7  6  3  2  1  0      time0
# 10  9  8  7  4  3  2  1
# 11 10  9  8  5  4  3  2
# 15 14 13 12  9  8  7  6
# 16 15 14 13 10  9  8  7
# 17 16 15 14 11 10  9  8
# 21 20 19 18 15 14 13 12
# 22 21 20 19 16 15 14 13
# 23 22 21 20 17 16 15 14      time8

bit_precision = 4
row_index = [9, 8, 7, 6, 3, 2, 1, 0]
kij_index = [0, 1, 2, 6, 7, 8, 12, 13, 14]


for in_ch in range(X.size(0)):  # big time loop - in_ch = 8
    filename = 'activation_os_ich' + str(in_ch) + '.txt'
    file = open(filename, 'w') #write to file
    file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
    file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
    file.write('#................#\n')
    
    for kij in range(9): # kij 0~8 - time (+ kij_index[0-8])
        for j in range(8): # row 7-0 (row_index[0-7])
            ind = row_index[j]+kij_index[kij]
            X_bin = '{0:04b}'.format(int(X[in_ch, ind].item()+0.001))
            for k in range(bit_precision):
                file.write(X_bin[k])        
            #file.write(' ')  # for visibility with blank between words, you can use
            print(ind, end=' ')
        file.write('\n')
        print("\n")
file.close() #close file 

9 8 7 6 3 2 1 0 

10 9 8 7 4 3 2 1 

11 10 9 8 5 4 3 2 

15 14 13 12 9 8 7 6 

16 15 14 13 10 9 8 7 

17 16 15 14 11 10 9 8 

21 20 19 18 15 14 13 12 

22 21 20 19 16 15 14 13 

23 22 21 20 17 16 15 14 

9 8 7 6 3 2 1 0 

10 9 8 7 4 3 2 1 

11 10 9 8 5 4 3 2 

15 14 13 12 9 8 7 6 

16 15 14 13 10 9 8 7 

17 16 15 14 11 10 9 8 

21 20 19 18 15 14 13 12 

22 21 20 19 16 15 14 13 

23 22 21 20 17 16 15 14 

9 8 7 6 3 2 1 0 

10 9 8 7 4 3 2 1 

11 10 9 8 5 4 3 2 

15 14 13 12 9 8 7 6 

16 15 14 13 10 9 8 7 

17 16 15 14 11 10 9 8 

21 20 19 18 15 14 13 12 

22 21 20 19 16 15 14 13 

23 22 21 20 17 16 15 14 

9 8 7 6 3 2 1 0 

10 9 8 7 4 3 2 1 

11 10 9 8 5 4 3 2 

15 14 13 12 9 8 7 6 

16 15 14 13 10 9 8 7 

17 16 15 14 11 10 9 8 

21 20 19 18 15 14 13 12 

22 21 20 19 16 15 14 13 

23 22 21 20 17 16 15 14 

9 8 7 6 3 2 1 0 

10 9 8 7 4 3 2 1 

11 10 9 8 5 4 3 2 

15 14 13 12 9 8 7 6 

16 15 14 13 10 9 8 7 

17 16 15 14 11 10 9 8 

21 20 19 18 15 14 13 12 

22 21 20 19 16 15 14 13 

23 22 

In [16]:
# weight stationary
tile_id = 0 
nij = 200 # just a random number
#X = a_tile[tile_id,:,nij:nij+64]  # [tile_num, array row num, time_steps]

bit_precision = 4
file = open('activation1.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X.size(1)):  # time step
    for j in range(X.size(0)): # row #
        X_bin = '{0:04b}'.format(int(X[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(X_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file 

In [17]:
X[:,7]

tensor([5., 0., 0., 5., 0., 0., 8., 0.], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [42]:
weight_int.size()

torch.Size([64, 3, 3, 3])

In [43]:
weight_int.size() # 64, 3, 3, 3
W = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))
W.size() # 64, 3, 9  # o_ch, in_ch, kij

torch.Size([64, 3, 9])

In [52]:
W[0][:][:]

tensor([[ 4.,  2.,  0.,  3., -1., -0., -1., -3., -3.],
        [ 2., -1., -3.,  2., -2., -2., -1., -4., -3.],
        [ 3.,  1., -2.,  4.,  2.,  1.,  1.,  0., -1.]], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [55]:
# output stationary
bit_precision = 4
file = open('weight_os.txt', 'w') #write to file
file.write('#kij0och7[msb-lsb],kij0och6[msb-lst],....,kij0och0[msb-lst]#\n')
file.write('#kij1och7[msb-lsb],kij1och6[msb-lst],....,kij1och0[msb-lst]#\n')
file.write('#................#\n')
for in_ch in range(W.size(1)):   # in_ch = 3
    for kij in range(W.size(2)):   # kij = 9
        for o_ch in range(8):   # o_ch = 64 -> only first 8
            if (W[7-o_ch, in_ch, kij].item()<0):
                W_bin = '{0:04b}'.format(int(W[7-o_ch, in_ch, kij].item()+2**bit_precision+0.001))
            else:
                W_bin = '{0:04b}'.format(int(W[7-o_ch, in_ch, kij].item()+0.001))
            for k in range(bit_precision):
                file.write(W_bin[k])        
            #file.write(' ')  # for visibility with blank between words, you can use
            print(W[7-o_ch, in_ch, kij].item(), end=' ')
        file.write('\n')
        print('\n')
file.close() #close file  


0.0 0.0 3.0 -1.0 -0.0 -4.0 -0.0 4.0 

0.0 0.0 3.0 -0.0 0.0 -2.0 0.0 2.0 

0.0 0.0 2.0 -1.0 -0.0 0.0 0.0 0.0 

0.0 0.0 3.0 -1.0 -0.0 -6.0 -1.0 3.0 

0.0 0.0 4.0 -1.0 -0.0 1.0 -1.0 -1.0 

0.0 0.0 2.0 -1.0 -0.0 5.0 -0.0 -0.0 

0.0 0.0 1.0 0.0 -0.0 -1.0 -2.0 -1.0 

0.0 0.0 1.0 -0.0 -0.0 3.0 -2.0 -3.0 

0.0 0.0 1.0 -0.0 -0.0 3.0 -2.0 -3.0 

0.0 0.0 1.0 -1.0 -0.0 -4.0 1.0 2.0 

0.0 0.0 0.0 -1.0 0.0 -1.0 1.0 -1.0 

0.0 0.0 -1.0 -1.0 -0.0 4.0 1.0 -3.0 

0.0 1.0 -0.0 -1.0 0.0 -7.0 0.0 2.0 

-0.0 1.0 0.0 -1.0 0.0 2.0 0.0 -2.0 

0.0 0.0 0.0 -0.0 -0.0 7.0 1.0 -2.0 

0.0 0.0 -2.0 1.0 0.0 -4.0 -1.0 -1.0 

0.0 0.0 -1.0 0.0 0.0 1.0 -2.0 -4.0 

0.0 0.0 -1.0 0.0 0.0 4.0 -2.0 -3.0 

0.0 1.0 1.0 -0.0 0.0 -3.0 1.0 3.0 

-0.0 1.0 1.0 -0.0 1.0 -1.0 1.0 1.0 

0.0 1.0 -1.0 -0.0 0.0 3.0 1.0 -2.0 

0.0 1.0 0.0 -0.0 1.0 -7.0 1.0 4.0 

0.0 1.0 -0.0 -0.0 1.0 1.0 1.0 2.0 

0.0 1.0 -1.0 -0.0 0.0 6.0 1.0 1.0 

0.0 1.0 -2.0 0.0 0.0 -4.0 -1.0 1.0 

0.0 1.0 -2.0 0.0 1.0 -1.0 -1.0 0.0 

0.0 1.0 -2.0 0.0 0.0 2.0 -1.0 -1.0 

In [21]:
# output stationary - separate files
bit_precision = 4
for in_ch in range(W.size(0)):   # in_ch = 8
    filename = 'weight_os_ich' + str(in_ch) + '.txt'
    file = open(filename, 'w') #write to file
    file.write('#kij0och7[msb-lsb],kij0och6[msb-lst],....,kij0och0[msb-lst]#\n')
    file.write('#kij1och7[msb-lsb],kij1och6[msb-lst],....,kij1och0[msb-lst]#\n')
    file.write('#................#\n')
    for kij in range(W.size(2)):   # kij = 9
        for o_ch in range(W.size(1)):   # o_ch = 8
            if (W[in_ch, 7-o_ch, kij].item()<0):
                W_bin = '{0:04b}'.format(int(W[in_ch, 7-o_ch, kij].item()+2**bit_precision+0.001))
            else:
                W_bin = '{0:04b}'.format(int(W[in_ch, 7-o_ch, kij].item()+0.001))
            for k in range(bit_precision):
                file.write(W_bin[k])        
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
file.close() #close file  


In [22]:
# weight stationary
bit_precision = 4
file = open('weight1.txt', 'w') #write to file
file.write('#col0row7[msb-lsb],col0row6[msb-lst],....,col0row0[msb-lst]#\n')
file.write('#col1row7[msb-lsb],col1row6[msb-lst],....,col1row0[msb-lst]#\n')
file.write('#................#\n')
for kij in range(9):
    for i in range(W.size(0)):  
        for j in range(W.size(1)):
            if (W[i, 7-j, kij].item()<0):
                W_bin = '{0:04b}'.format(int(W[i,7-j,kij].item()+2**bit_precision+0.001))
            else:
                W_bin = '{0:04b}'.format(int(W[i,7-j,kij].item()+0.001))
            for k in range(bit_precision):
                file.write(W_bin[k])        
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
file.close() #close file  


In [23]:
W[0,:,0]

tensor([-7.0000,  3.0000,  7.0000, -2.0000, -3.0000, -3.0000,  7.0000,  2.0000],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [24]:
p_nijg = range(X.size(1)) ## psum nij group

In [25]:
psum = torch.zeros(8, len(p_nijg), 9).cuda() 

In [26]:
for kij in range(9):  
    for nij in p_nijg:       # time domain, sequentially given input
        m = nn.Linear(8, 8, bias=False)
        m.weight = torch.nn.Parameter(W[:,:,kij])
        psum[:, nij, kij] = m(X[:,nij]).cuda()

In [27]:
bit_precision = 16
file = open('psum1.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')
for kij in range(9):
    for i in range(psum.size(1)):
        for j in range(psum.size(0)):
            if (psum[7-j,i,kij].item()<0):
                P_bin = '{0:016b}'.format(int(psum[7-j,i,kij].item()+2**bit_precision+0.001))
            else:
                P_bin = '{0:016b}'.format(int(psum[7-j,i,kij].item()+0.001))
            for k in range(bit_precision):
                file.write(P_bin[k])
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
file.close()

In [28]:
psum[:,8,0]

tensor([ 22.,  67.,  25., -42., -53., -38.,  31.,  85.], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [57]:
output_int.size()

torch.Size([128, 64, 32, 32])

In [58]:
out = output_int[0]

In [60]:
out = torch.reshape(out, (out.size(0), -1))

In [32]:
out = relu(out)

In [62]:
out.size() # och, onij

torch.Size([64, 1024])

In [63]:
bit_precision = 16
file = open('output_os.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')

for i in range(8): # first 8 och
    for j in range(8): # first 8 onij
        if (out[7-j,i].item()<0):
            O_bin = '{0:016b}'.format(int(out[7-j,i].item()+2**bit_precision+0.001))
        else:
            O_bin = '{0:016b}'.format(int(out[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(O_bin[k])
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close()

In [64]:
out[:,4]

tensor([ 126.0000,  168.0000,  -76.0000,  -36.0000,   56.0000,  -28.0000,
        -100.0000,    0.0000,   50.0000,  112.0000,  -22.0000,  -24.0000,
          78.0000,  -72.0000,  -30.0000,   50.0000,    0.0000,  -60.0000,
          72.0000, -134.0000,  -60.0000,   28.0000,  238.0000,  -28.0000,
          70.0000,    0.0000,    0.0000,   70.0000,  -60.0000,    0.0000,
         188.0000,  -12.0000,    2.0000,  -50.0000,   12.0000,  -74.0000,
          68.0000,   64.0000,   14.0000,  134.0000, -266.0000,  -20.0000,
        -230.0000,  -44.0000,   56.0000,  -74.0000,  -14.0000,   42.0000,
         -38.0000,   22.0000,  366.0000,   42.0000,  132.0000,  -54.0000,
          -6.0000,  116.0000,   44.0000,  140.0000,  -74.0000, -134.0000,
          34.0000,  -10.0000,    0.0000,   80.0000], device='cuda:0',
       grad_fn=<SelectBackward0>)